In [1]:
print("GRACE")

GRACE


In [ ]:
from PIL import Image
import os

def images_to_pdf(image_folder, output_pdf_path):
    """
    Combine all images in a folder into a single PDF with 4 images per page,
    maintaining their aspect ratio, without reducing resolution, and adding thick white borders.

    :param image_folder: Path to the folder containing images.
    :param output_pdf_path: Path to save the generated PDF.
    """
    # Get a list of all image files in the folder
    image_files = [
        os.path.join(image_folder, file)
        for file in sorted(os.listdir(image_folder))
        if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif"))
    ]

    if not image_files:
        print("No images found in the folder.")
        return

    # Define page size and layout for 4 images per page
    page_width, page_height = 595, 842  # A4 size in points (72 DPI)
    max_width, max_height = page_width // 2, page_height // 2  # Maximum size for each image including borders
    border_thickness = 20  # Thickness of the white border

    pages = []
    page = Image.new("RGB", (page_width, page_height), "white")
    coords = [(0, 0), (page_width // 2, 0), (0, page_height // 2), (page_width // 2, page_height // 2)]
    current_image_index = 0

    for image_file in image_files:
        img = Image.open(image_file).convert("RGB")

        # Maintain aspect ratio while scaling dimensions
        img_ratio = img.width / img.height
        if img.width > img.height:
            scale = min((max_width - 2 * border_thickness) / img.width, (max_height - 2 * border_thickness) / img.height)
        else:
            scale = min((max_width - 2 * border_thickness) / img.width, (max_height - 2 * border_thickness) / img.height)

        new_width = int(img.width * scale)
        new_height = int(img.height * scale)

        # Create a blank canvas larger than the image to include borders
        img_canvas = Image.new("RGB", (max_width, max_height), "white")
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Center the scaled image on the canvas with borders
        x_offset = (max_width - new_width) // 2
        y_offset = (max_height - new_height) // 2
        img_canvas.paste(img, (x_offset, y_offset))

        # Paste the canvas onto the page
        x, y = coords[current_image_index % 4]
        page.paste(img_canvas, (x, y))
        current_image_index += 1

        # Save page when 4 images are added or last image is processed
        if current_image_index % 4 == 0 or current_image_index == len(image_files):
            pages.append(page)
            if current_image_index != len(image_files):
                page = Image.new("RGB", (page_width, page_height), "white")

    # Save all pages into a single PDF
    if pages:
        pages[0].save(output_pdf_path, save_all=True, append_images=pages[1:])
        print(f"PDF created successfully at {output_pdf_path}")

# Example usage
image_folder = "uploads/32"  # Path to the folder containing images
output_pdf = "output.pdf"  # Path to save the generated PDF

images_to_pdf(image_folder, output_pdf)


PDF created successfully at output.pdf
